In [18]:
from numcosmo_py import Nc, Ncm, sky_match
Ncm.cfg_init()

from astropy.table import Table, join, unique
import pandas as pd

import sys
sys.path.insert(0, "/global/homes/c/cinlima/gcrcatalogs-new/gcr-catalogs")
import GCRCatalogs
print('GCRCatalogs =', GCRCatalogs.__version__, '|' ,'GCR =', GCRCatalogs.GCR.__version__)

%load_ext autoreload
%autoreload 2

GCRCatalogs = 1.10.2 | GCR = 0.9.2
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Catalogs

In [2]:
# #Cosmo DC2
# cosmodc2 = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image')

# #Cosmo DC2 + RedMaPPer
# catalog_dc2_redmapper = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_redmapper_v0.8.1')

# print("CosmoDC2 info:")
# print(f"Version: {cosmodc2.get_catalog_info('version')}")
# print(f"Sky area: {cosmodc2.get_catalog_info('sky_area')}")
# print(f"Cosmology: {cosmodc2.get_catalog_info('cosmology')}")

# quantities = catalog_dc2_redmapper.list_all_quantities()
# cluster_quantities = [q for q in quantities if 'member' not in q]
# member_quantities = [q for q in quantities if 'member' in q]
    
# cluster_data = Table(catalog_dc2_redmapper.get_quantities(cluster_quantities)) # cDC2 + RM cluster data
# member_data = Table(catalog_dc2_redmapper.get_quantities(member_quantities)) # cDC2 + RM member data

# # Restrictions on mass and redshift
# min_halo_mass = 1e13 #Msun
# redshift_max = 1.2

# quantities_wanted = ['redshift','halo_mass','halo_id','galaxy_id','ra','dec', 'is_central']
# query = GCRCatalogs.GCRQuery('(halo_mass > ' + str(min_halo_mass) +') & (redshift < ' + str(redshift_max) +') & (is_central < ' + str(redshift_max) +') ')
# truth_data = Table(cosmodc2.get_quantities(quantities_wanted, [query]))

# table_halos = truth_data[truth_data['is_central'] == True]

CosmoDC2 info:
Version: 1.1.4
Sky area: 439.78987
Cosmology: {'H0': 71.0, 'Om0': 0.2648, 'Ob0': 0.0448, 'sigma8': 0.8, 'n_s': 0.963}


In [3]:
# CLUSTER DATA

cluster_data = Table.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CatalogsMatching/cluster_data.fits')

cluster_member_data = Table.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CatalogsMatching/member_data.fits')
cluster_member_data.rename_column('cluster_id_member', 'cluster_id')


In [4]:
# HALO DATA

truth_data = Table.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CatalogsMatching/truth_data.fits')

truthdf = truth_data.to_pandas()

hascentral = truthdf[truthdf['is_central'] ==  True]
halo_central_ids = list(hascentral['halo_id'].unique())
central_position = hascentral[['ra', 'dec', 'redshift', 'halo_id']].rename(columns = {'ra': 'halo_ra', 'dec': 'halo_dec', 'redshift': 'halo_redshift'})
halos_dataframe = pd.merge(truthdf[truthdf['halo_id'].isin(halo_central_ids)], central_position, how='inner', on=['halo_id'])


halo_member_data = Table.from_pandas(halos_dataframe)
halo_data =  unique(halo_member_data['halo_id', 'halo_mass', 'halo_ra', 'halo_dec', 'halo_redshift'])

## Matching

In [5]:
halo_coordinates = {"RA":"halo_ra" , "DEC":"halo_dec" , "z":"halo_redshift"}
halo_ids  = {"ID": "halo_id", "MemberID": "galaxy_id"}

detections_coordinates =  {"RA":"cluster_ra" , "DEC":"cluster_dec" , "z":"cluster_redshift"}
detections_ids  = {"ID": "cluster_id", "MemberID": "id_member"}


halos = sky_match.SkyMatch(
    query_data= halo_data,
    query_coordinates=halo_coordinates,
    query_member_data=halo_member_data,
    query_ids = halo_ids, 
    match_data=cluster_data,
    match_coordinates=detections_coordinates,
    match_member_data=cluster_member_data,
    match_ids = detections_ids
)


detections = halos.invert_query_match()

In [6]:
%time halos_matched = halos.match_ID()

CPU times: user 14.5 s, sys: 492 ms, total: 15 s
Wall time: 15 s


In [7]:
%time best = halos_matched.select_best()

CPU times: user 5.94 s, sys: 81 ms, total: 6.03 s
Wall time: 6.03 s


In [8]:
%time best_table = halos_matched.to_table_best(best)

CPU times: user 8.39 ms, sys: 73 μs, total: 8.46 ms
Wall time: 7.65 ms


In [9]:
# best_table

In [36]:
%time complete_table = halos_matched.to_table_complete()

CPU times: user 10.4 s, sys: 135 ms, total: 10.5 s
Wall time: 10.5 s


In [38]:
# complete_table

In [32]:
%time detections_matched = detections.match_ID()

CPU times: user 11.3 s, sys: 465 ms, total: 11.8 s
Wall time: 11.6 s


In [13]:
%time detections_best = detections_matched.select_best()

CPU times: user 5.2 s, sys: 58.4 ms, total: 5.26 s
Wall time: 5.27 s


In [14]:
%time detections_table_best = detections_matched.to_table_best(detections_best)

CPU times: user 3.48 ms, sys: 495 μs, total: 3.97 ms
Wall time: 3.32 ms


In [15]:
#detections_table_best

In [33]:
%time complete_table_detections = detections_matched.to_table_complete()

CPU times: user 2.63 s, sys: 67.7 ms, total: 2.7 s
Wall time: 2.7 s


In [39]:
# complete_table_detections